In [69]:
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2
from skimage import data
  
'''Podajemy wspolrzedne dwoch punktow, algorytm oblicza dlugosc odcinka, ktory tworza.
Dziala tylko na liczbach calkowitych
image-obraz wejsciowy
xa,ya,xb,yb-wspolrzedne koncow odcinka
w,h-srodek obrazu
inv=True-stosujemy algorytm dla odwrotnej transformaty Radona'''
  
def bresenham(image,xa,xb,ya,yb,w,h,inv=False,col=1):
    points=[]
    if xa<xb:
        x=1
        dx=xb-xa;
    else:
        x=-1
        dx=xa-xb

    if ya<yb:
        y=1
        dy=yb-ya
    else:
        y=-1
        dy=ya-yb  
        
    if xa>=0 and xa<w and ya>=0 and ya<h:
        if inv==False:
            color = image[h-1-ya][xa]
            if color > 0: points.append(color)
        else:
            image[h-1-ya][xa] += col
    if dx>dy:
        a=(dy-dx)*2
        b=dy*2
        d=b-dx
        while xa!=xb:
            if d>=0:
                xa+=x
                ya+=y
                d+=a
            else:
                d+=b
                xa+=x
            if xa>=0 and xa<w and ya>=0 and ya<h:
                if inv==False:
                    color = image[h-1-ya][xa]
                    if color > 0: points.append(color)
                else:
                    image[h-1-ya][xa] += col
    else:
        a=(dx-dy)*2
        b=dx*2
        d=b-dy
        while ya!=yb:
            if d>=0:
                xa+=x
                ya+=y
                d+=a
            else:
                d+=b
                ya+=y
            if xa>=0 and xa<w and ya>=0 and ya<h:
                if inv==False:
                    color = image[h-1-ya][xa]
                    if color > 0: points.append(color)
                else:
                    image[h-1-ya][xa] += col
    if inv==False:
        return points
    else:
        return image

In [70]:
#rysuje obraz po transformacie
def drawRadon(img):
    pom=img/max(img.flatten())
    cv2.imshow('sinogram',pom)
    cv2.waitKey(1000)
    
def drawInvRadon(img):
    img -= min(img.flatten())
    pom =img/ max(img.flatten())
    cv2.imshow('obraz końcowy',pom)
    cv2.waitKey(1000)

In [75]:
'''Petla wyznacza kolejne punkty, dla ktorych stosowany jest algorytm  bresenhama.
Zastosowano model emiterow/detektorow rownolegly'''
def radonRepeat(inputImg,outputImg,w,h,nsteps,alfa,ndetectors=360,l=360,step=10,inv=False):
    #odleglosc miedzy dwoma detektorami
    detectorsDistance=l/(ndetectors-1)
    r=math.sqrt(w*w+h*h) #promien okregu opisanego
    alfa=alfa*math.pi/180 #kat o ktory sie przesuwamy w radianach
    
    for i in range(nsteps):
        angle=alfa*i
        for j in range(0,ndetectors):
            d=l/2-detectorsDistance*j
            dsign=1
            if d<0:
                dsign=-1
                d=abs(d)
            
            sinus=abs(math.sin(angle))
            cosinus=abs(math.cos(angle))
            tangens=abs(math.tan(angle))
            a=math.sqrt(r*r-d*d)
            b=0
            if sinus<0.0001:
                ya=int(a+h)
                yb=int(h-a)
                if dsign==1:
                    xa=int(w+d)
                    xb=int(w+d)
                else:
                    xa=int(w-d)
                    xb=int(w-d)
            else:
                b=d/tangens
                y2=d/sinus
                
                if angle<=1.57:
                    if dsign==1:
                        xa=int(w+(a-b)*sinus)
                        xb=int(w-(a+b)*sinus)
                        y1=int((a-b)*cosinus)
                        ya=int(y1+y2+h)
                        y1=int((a+b)*cosinus)
                        yb=int(h-(y1-y2))
                    else:
                        xa=int(w+(a+b)*sinus)
                        xb=int(w-(a-b)*sinus)
                        y1=int((a+b)*cosinus)
                        ya=int(y1-y2+h)
                        y1=int((a-b)*cosinus)
                        yb=int(h-y1-y2)
                else:
                    if dsign==1:
                        xa=int(w+(a+b)*sinus)
                        xb=int(w-(a-b)*sinus)
                        y1=int((a+b)*cosinus)
                        ya=int(h-(y1-y2))
                        y1=int((a-b)*cosinus)
                        yb=int(h+y1+y2)
                    else:
                        xa=int(w+(a-b)*sinus)
                        xb=int(w-(a+b)*sinus)
                        y1=int((a-b)*cosinus)
                        ya=int(h-y1-y2)
                        y1=int((a+b)*cosinus)
                        yb=int(h+y1-y2)
            
            if inv==False:
                points=bresenham(inputImg,xa,xb,ya,yb,inputImg.shape[1],inputImg.shape[0])
                if len(points)>0:
                   outputImg[j][i]=sum(points) 
            else:
                color=inputImg[j,i]
                outputImg=bresenham(outputImg,xa,xb,ya,yb,inputImg.shape[0],inputImg.shape[0],True,color)
        if i%step==0:
            if inv==False:
                drawRadon(outputImg)
            else:
                drawInvRadon(outputImg)           
    if inv==False:
        drawRadon(outputImg)
    else:
        drawInvRadon(outputImg)           
    return outputImg

In [80]:
'''
model rownolegly
image-obraz wejsciowy
alfa-kat, o ktory obracamy emiter/detektor (w stopniach)
ndetectors-liczba detektorow
l-rozpietosc detektorow
sinogram-obraz wyjsciowy
 '''
def radon(image,alfa=1,ndetectors=360,l=360,step=10): 
    #w,h- srodek zdjecia
    w=image.shape[1]//2
    h=image.shape[0]//2 
    nsteps=int(180/alfa)
    sinogram = np.zeros((ndetectors,nsteps+1))

    sinogram=radonRepeat(image,sinogram,w,h,nsteps,alfa,ndetectors,l,step)
    
    #normalizacja
    sinogram /= max(sinogram.flatten())
    return sinogram

def inverseRadon(sinogram,alfa=1,ndetectors=360,l=360,step=10):
    h=sinogram.shape[0]//2
    w=h
    nsteps=int(180/alfa+1)
    image = np.zeros((sinogram.shape[0],sinogram.shape[0]))

    image=radonRepeat(sinogram,image,w,h,nsteps,alfa,ndetectors,l,step,True)

    #normalizacja
    image -= min(image.flatten())
    image /= max(image.flatten())

    return image

In [81]:
image = cv2.imread("Kwadraty2.jpg", 0)

#dodaje czarne piksele tworzac kwadratowe zdjecie
test=np.zeros(len(image[0]))
if image.shape[1]>image.shape[0]:
    test=np.zeros((image.shape[1],image.shape[1]))
    for i in range(len(image)):
        test[i]=image[i]
else:
    test=np.zeros((image.shape[0],image.shape[0]))
    for i in range(len(image)):
        for j in range(len(image[0])):
            test[i][j]=image[i][j]


In [132]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

def slider(x):
    return x

#liczba detektorow
caption = widgets.Label(value='Liczba detektorów:')
n = interactive(slider, x=widgets.IntSlider(value=int(len(test)*0.4), description='n', max=int(len(test)*0.8), min=int(len(test)*0.2)))
display(caption,n)
#n=n.result

#rozpietosc detektorow  
caption = widgets.Label(value='Rozpiętość detektorów (dla l<n za l przyjmowana jest liczba detektorów):')
l = interactive(slider, x=widgets.IntSlider(value=int(len(test)*0.7), description='l', max=int(len(test)*0.95), min=(int(len(test)*0.45))))
display(caption,l)
#l=l.result

#krok alfa układu detektorów
caption = widgets.Label(value='Krok alfa układu emiter/detektor:')
alfa = interactive(slider, x=widgets.FloatSlider(value=1, description='alfa', max=2, min=0.5, step=0.5))
display(caption,alfa)
#alfa=alfa.result

#co ile kroków wyświetlamy obraz wynikowy
caption = widgets.Label(value='Co ile kroków wyświetlany jest tworzony obraz:')
step = interactive(slider, x=widgets.IntSlider(value=10, description='krok', max=90, min=2))
display(caption,step)
#step=step.result

button = widgets.Button(description="Wygeneruj obraz!")
display(button)


def mainn(b):
    n2=n.result
    l2=l.result
    alfa2=alfa.result
    step2=step.result
    #print(n2,l2,alfa2,step2)
    if l2<n2: l2=n2
    #wykonanie radona
    radonSin=radon(test,alfa2,n2,l2,step2)
    #l-czyli rozpietosc zmienia sie proporcjonalnie do zmiany wielkosci obrazu 
    l3=n2*l2/len(test)
    radonInv=inverseRadon(radonSin,alfa2,n2,l3,step2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
button.on_click(mainn)

Label(value='Liczba detektorów:')

interactive(children=(IntSlider(value=153, description='n', max=307, min=76), Output()), _dom_classes=('widget-interact',))

Label(value='Rozpiętość detektorów (dla l<n za l przyjmowana jest liczba detektorów):')

interactive(children=(IntSlider(value=268, description='l', max=364, min=172), Output()), _dom_classes=('widget-interact',))

Label(value='Krok alfa układu emiter/detektor:')

interactive(children=(FloatSlider(value=1.0, description='alfa', max=2.0, min=0.5, step=0.5), Output()), _dom_classes=('widget-interact',))

Label(value='Co ile kroków wyświetlany jest tworzony obraz:')

interactive(children=(IntSlider(value=10, description='krok', max=90, min=2), Output()), _dom_classes=('widget-interact',))

Button(description='Wygeneruj obraz!', style=ButtonStyle())